<a href="https://colab.research.google.com/github/MitchellKrops/PythonAutomation/blob/main/Level_Product_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Importeerassistent voor maatschappij producten
## Importeer XML lib
import xml.etree.ElementTree as ET
from tabulate import tabulate

try:
  Bron_Bestand = 'test.xml' ##input("Geef bronbestand in:")
  tree = ET.parse(Bron_Bestand)
  root = tree.getroot()
  print("-----------------------------------------------------------------------------")
  ## Uitlezen brongegevens
  for info in root.iter('INFO'):
      waarde = info.text

  for Naam_EXTPRD in root.iter("NM_EXTPRD"):
    Externe_productomschrijving = Naam_EXTPRD.text

  infotext = waarde.split("-")
  Bron_Maatschappij = infotext[1]
  Soort_Product = infotext[2]
  Productcode = infotext[3]
  
  ## Tonen brongegevens
  print()
  print("Het bronproduct bevat de volgende gegevens:")
  print()
  info_data = [[Bron_Maatschappij, Soort_Product, Productcode, Externe_productomschrijving]]
  print (tabulate(info_data, headers=["Maatschappij", "Soort product", "Productcode","Externe productomschrijving"],numalign="left"))
  print()

  Branchecodes_list = []
  EXTBRA_list = []
  
  MIJ_SRT_VRZ = root.findall('.//MIJ_SRT_VRZ')
  for MIJ_SRT in MIJ_SRT_VRZ :
    naam = MIJ_SRT.find('NM_EXTBRA').text
    Branchecodes_list.append(naam)
    br_code = MIJ_SRT.find('KD_BR_VAN').text
    EXTBRA_list.append(br_code)
 
  branche_data = zip(EXTBRA_list,Branchecodes_list)
  print (tabulate(branche_data, headers=["Branchecode","Externe brancheomschrijving"],numalign="left"))
  print()
  print("-----------------------------------------------------------------------------")

  ## Input doelgegevens
  print()
  while True:
    Doel_Maatschappij = input("Geef doel maatschappij in:")
    if not len(Doel_Maatschappij) <= 5:
      print("**Fout: Maatschappijnaam mag maximaal 5 karakters lang zijn**")
      continue
    else:

      break  

  while True:  
    Doel_Productcode: str = input("Geef doel productcode in:")
    if len(Doel_Productcode) > 3 or Doel_Productcode.isdecimal() == False:
      print("**Fout: Productcode mag maximaal 3 cijfers lang zijn**")
      continue
    else:

      break 
  print()
  Naam_EXTPRD_keuze = input("Wil je de externe productomschrijving? aanpassen? y/n:")
  if Naam_EXTPRD_keuze == "y":
    Naam_EXTPRD_nieuw = input("Geef nieuwe externe productomschrijving in:")
    for element in root.iter('NM_EXTPRD'):
      element.text = Naam_EXTPRD_nieuw

  print()  
  
  ## Aanmaken nieuw INFO Attribute
  Info_Nieuw = "-{0}-{1}-{2}".format(Doel_Maatschappij, Soort_Product, Doel_Productcode)

  ## Wegschrijven nieuw INFO Attribute
  for element in root.iter('INFO'):
      element.text = Info_Nieuw

  for productcode in root.iter('KD_PROD'):
      productcode.text = productcode.text.replace(Productcode, Doel_Productcode)

  for element in root.iter():
      try:
          element.text = element.text.replace(Bron_Maatschappij, Doel_Maatschappij)
      except AttributeError:
          pass

  ## Vastleggen en tonen branchecodes / dekkingscodes

  Branchecodes_keuze = input("Wil je branchecodes aanpassen? y/n:")
  print()
  Branchecodes_list_nieuw = []

  if Branchecodes_keuze == "y":
      for i in Branchecodes_list:
          
          while True:
            branchecode_nieuw = input("Branchcode "+ i + " vervangen door:")
            if len(branchecode_nieuw) > 3 or branchecode_nieuw.isdecimal() == False:
              print("**Fout: Branchecode mag maximaal 3 cijfers lang zijn**")
              continue
            else:

              break
          Branchecodes_list_nieuw.append(branchecode_nieuw)
      print()
      print("De nieuwe branchecodes zijn:")
      print(Branchecodes_list_nieuw)

       ## Vervangen van brongegevens voor doelgegevens
      Branchecodes_rubrieken = ['KD_BR_TOT', 'KD_BR_VAN', 'KD_BR_ELM']
      for i in range(len(Branchecodes_rubrieken)):
          for element in root.iter(Branchecodes_rubrieken[i]):
              index = Branchecodes_list.index(element.text)
              element.text = Branchecodes_list_nieuw[index]
              
  Branchecode_omschrijving_keuze = input("Wil je de externe brancheomschrijving aanpassen? y/n:")  

  ## Informatie wegschrijven in Export.xml
  tree.write("{0}-{1}-{2}.xml".format(Doel_Maatschappij, Soort_Product, Doel_Productcode), method='xml', encoding='utf-8', xml_declaration=True)
  print()
  print("Succesvol importbestand aangemaakt: " + Doel_Maatschappij + "-" + Soort_Product + "-" + Doel_Productcode + ".xml")

except KeyboardInterrupt:
  print("Stop")
